## 16S Amplicon Demultiplex Workflow

* Chenghao Zhu
* 2018/10/29
* chhzhu@ucdavis.edu

This workflow covers the basic steps of processing 16S sequencing data, including demultiplex, filtering, and trimming primers for the raw fastq files. The demultiplexed data is then ready for otu picking. This workflow is designated for the old sequencing method from David [Miles lab](http://mills.ucdavis.edu/), which only use barcodea on the forward primer (that means the reverse (downstream) primer is not barcoded). 

In this workflow, the raw paired end read fastq files were first demultiplexed using the barcode to pick up reads that have barcode in the begining of R1. Then the unmatched reads (unmatched_R1.fastq, unmatched_R2.fastq) were demultiplexed using barcode as reverse barcode, to pick up reads that have barcode in the begining of R2. The command line tool [**fastq_multx**](https://github.com/brwnj/fastq-multx) is used to demultiplex the sequencing reads without merging then (one example that merge while demultiplexing is [PEAR](https://sco.h-its.org/exelixis/web/software/pear/doc.html)). The demultiplexed reads (sample01_R1.fastq, sample-1_R2.fastq, ...) were then filtered using a python script, to remove reads that don't have the primers in the right place (most likely generated because of errors). Primers were then cut off from each end by specifying the lengths of priimers, and the 2 fastq files that belong to the same sample were concatenated together. In the very last step, FastQC is used to check the quality of reads, to determine the length to use in [**DADA2**](https://benjjneb.github.io/dada2/tutorial.html).

This workflow requires around 40G disk space. The actual disk space might vary depands on the sample size. Make sure your hvae at lease **50G** of empty disk space before you start.

This workflow is writen in Jupyter notebook. If you want to run directly in shell command, please remove the "!" in front of each command. The "!" is a trick in Jupyter Notebook to exacute shell commands.


**Prerequisite tools**: 

- [**fastq-multx**](https://github.com/brwnj/fastq-multx):
        Can be installed using the following command:
        
        $ conda install -c bioconda fastq-multx

- [**paired_end_reads_filter_by_primer.py**](https://zhuchcn.github.io/docs/workflows/demultx/paired_end_reads_filter_by_primer.py):
        This script has to be put under the same directory as your jupyter notebook. Ask Trevor (chhzhu@ucdavis.edu) for this script.
        Biopython is also required to run this script successfully. It can be installed using:
        
        $ conda install bio

- [**fastqc**](http://www.bioinformatics.babraham.ac.uk/projects/fastqc/):
        Is you are using brew, you can use:
        brew install fastqc
        If you can't install fastqc, contact Trevor (chhzhu@ucdavis.edu)

- [**fastx-toolkit**](http://hannonlab.cshl.edu/fastx_toolkit/):
        Can be installed using conda:
        
        $ conda install -c bioconda fastx-toolkit

### Step 1. Demultiplex the forward

16S sequencing method mixes samples, sometimes from different studies, into a pool that they call it "library", after PCR amplication. Samples are labeled using unique 6-digit (sometimes 8-digit) barcode previous to pooling. And those barcodes are what used to match the sequencing reads to its co-responding sample ID. And this process is called *demultiplexing*.

Many popular demultiplexing tools like [PEAR](https://sco.h-its.org/exelixis/web/software/pear/doc.html) merges the forward and reversed reads together while demultiplexing. A consequence of that is that the quality scores are all removed. The popular otu clustering algorithm [DADA2](https://benjjneb.github.io/dada2/tutorial.html) however uses the quality scores to remove noises. 

So here we use the command line tool [fastq-multx](https://github.com/brwnj/fastq-multx) to demultiplx and keep the forward and reversed read separated. Read the fastq-multx documentation for more information.

<div class="alert alert-block alert-warning">
<b>Warning:</b> This command generates around 17G fastq files for 40 samples. Make sure you enough disk space..
</div>

In [ ]:
!fastq-multx -h

In [ ]:
!mkdir -p demultx_R1
!mkdir -p demultx_R2

In [ ]:
!fastq-multx -B 2017_AZ_barcodes_FF.txt -m 0 -x -b\
            FFUBS-Run_S1_L001_R1_001.fastq \
            FFUBS-Run_S1_L001_R2_001.fastq \
            -o demultx_R1/%_R1.fastq \
            -o demultx_R1/%_R2.fastq

### Step 2. Demultiplex the reversed

<div class="alert alert-block alert-warning">
<b>Warning:</b> This command geneerates additional 21G fastq files for 40 samples.
</div>

In [ ]:
!fastq-multx -B /2017_AZ_barcodes_FF.txt -m 0 -x -b\
            demultx_R1/unmatched_R2.fastq \
            demultx_R1/unmatched_R1.fastq \
            -o demultx_R2/%_R2.fastq \
            -o demultx_R2/%_R1.fastq

<div class="alert alert-block alert-success">
<b>Tip:</b> The raw fastq fiels can be deleted now to save your disk space. As well as the unmatched_R1.fastq and unmatched_R2.fastq.
</div>

### Step 3. Filter

Although step 1 and 2 pick up sequences that only starts with the barcodes for each sample. However, some sequences that have barcodes at the begining, don't have primer right after, or don' have the reverse primer at the begining of the other read in the pair. The purpose of this step is filter out those reads, and only keep the reads that not only have barcodes, but also have both forward and reverse primer at the correct location of the sequences.

<div class="alert alert-block alert-warning">
<b>Warning:</b> Make sure the paired_end_reads_filter_by_primer.py file is at the correct place.
</div>

In [ ]:
!mkdir -p filt_demultx_R1
!mkdir -p filt_demultx_R2

In [ ]:
!ls demultx_R1/FF*_R1.fastq | cut -f2 -d '/' |cut -f1 -d '.' >filt_R1.txt
!ls demultx_R1/FF*_R2.fastq | cut -f2 -d '/' |cut -f1 -d '.' >filt_R2.txt

In [ ]:
!python paired_end_reads_filter_by_primer.py \
    --input-forward-list filt_R1.txt \
    --input-reverse-list filt_R2.txt \
    --input-path demultx_R1 \
    --output-path filt_demultx_R1 \
    --barcodes 2017_AZ_barcodes_FF.txt \
    --forward-primer GTGTGCCAGCMGCCGCGGTAA \
    --reverse-primer GGACTACNVGGGTWTCTAAT

In [ ]:
!python paired_end_reads_filter_by_primer.py \
    --input-forward-list filt_R2.txt \
    --input-reverse-list filt_R1.txt \
    --input-path demultx_R2 \
    --output-path filt_demultx_R2 \
    --barcodes 2017_AZ_barcodes_FF.txt \
    --forward-primer GTGTGCCAGCMGCCGCGGTAA \
    --reverse-primer GGACTACNVGGGTWTCTAAT

### Step 4. Trim off primers

In [ ]:
!ls filt_demultx_R1/FF*_R1.filt.fastq | cut -f2 -d '/' |cut -f1 -d '.' >trim_R1.txt
!ls filt_demultx_R1/FF*_R2.filt.fastq | cut -f2 -d '/' |cut -f1 -d '.' >trim_R2.txt

In [ ]:
!mkdir trim_demultx_R1
!mkdir trim_demultx_R2

In [ ]:
%%bash
while read trim
do
        fastx_trimmer -f 30 -i filt_demultx_R1/$trim.filt.fastq -o trim_demultx_R1/$trim.trim.fastq
done < trim_R1.txt

In [ ]:
%%bash
while read trim
do
        fastx_trimmer -f 21 -i filt_demultx_R1/$trim.filt.fastq -o trim_demultx_R1/$trim.trim.fastq
done < trim_R2.txt

In [ ]:
%%bash
while read trim
do
        fastx_trimmer -f 21 -i filt_demultx_R2/$trim.filt.fastq -o trim_demultx_R2/$trim.trim.fastq
done < trim_R1.txt

In [ ]:
%%bash
while read trim
do
        fastx_trimmer -f 30 -i filt_demultx_R2/$trim.filt.fastq -o trim_demultx_R2/$trim.trim.fastq
done < trim_R2.txt

### Step 5. Concatenate

This step is only 

In [ ]:
!mkdir -p alldemultx

In [ ]:
!ls trim_demultx_R1/FF*.fastq | cut -f2 -d '/' |cut -f1 -d '_' > sample_list.txt

In [ ]:
%%bash
while read id
do
        cat trim_demultx_R1/${id}_R1.trim.fastq trim_demultx_R2/${id}_R1.trim.fastq > alldemultx/${id}_R
done < sample_list.txt

In [ ]:
%%bash
while read id
do
        cat trim_demultx_R1/${id}_R2.trim.fastq trim_demultx_R2/${id}_R2.trim.fastq > alldemultx/${id}_R
done < sample_list.txt

### Step 6. FastQC

In [ ]:
!mkdir -p fastqc

In [ ]:
!cat alldemultx/FF*_R1.combo.fastq > R1.all.fastq
!cat alldemultx/FF*_R2.combo.fastq > R2.all.fastq

In [ ]:
!fastqc R1.all.fastq R2.all.fastq -o fastqc

In [ ]:
!rm R1.all.fastq
!rm R2.all.fastq

In [ ]:
!ls fastqc